In [1]:
import pandas as pd
from datetime import datetime
import ta
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv("niftybank.csv")
data.head()

,date,open,high,low,close,volume
0,2017-07-17 09:14:00,24011.40,24011.40,24011.40,24011.40,0
1,2017-07-17 09:15:00,24011.40,24014.40,23974.95,23974.95,0
2,2017-07-17 09:16:00,23975.45,23981.55,23963.15,23978.95,0
3,2017-07-17 09:17:00,23977.25,23995.30,23976.00,23984.60,0
4,2017-07-17 09:18:00,23983.25,23989.95,23983.25,23987.10,0


In [3]:
data["Date"] = data["date"].str.split(" ").str[0]
data["Time"] = data["date"].str.split(" ").str[1].str.split("+").str[0]
data["Date_Time"] = data['Date'] + " " + data["Time"]
data.drop("Date", axis = 1, inplace = True)
data



,date,open,high,low,close,volume,Time,Date_Time
0,2017-07-17 09:14:00,24011.40,24011.40,24011.40,24011.40,0,09:14:00,2017-07-17 09:14:00
1,2017-07-17 09:15:00,24011.40,24014.40,23974.95,23974.95,0,09:15:00,2017-07-17 09:15:00
2,2017-07-17 09:16:00,23975.45,23981.55,23963.15,23978.95,0,09:16:00,2017-07-17 09:16:00
3,2017-07-17 09:17:00,23977.25,23995.30,23976.00,23984.60,0,09:17:00,2017-07-17 09:17:00
4,2017-07-17 09:18:00,23983.25,23989.95,23983.25,23987.10,0,09:18:00,2017-07-17 09:18:00
...,...,...,...,...,...,...,...,...
616014,2024-03-07 15:25:00,47815.65,47815.65,47788.80,47802.45,0,15:25:00,2024-03-07 15:25:00
616015,2024-03-07 15:26:00,47805.65,47805.65,47790.45,47795.80,0,15:26:00,2024-03-07 15:26:00
616016,2024-03-07 15:27:00,47797.10,47802.95,47782.00,47800.35,0,15:27:00,2024-03-07 15:27:00
616017,2024-03-07 15:28:00,47802.70,47806.05,47783.75,47785.70,0,15:28:00,2024-03-07 15:28:00


In [4]:
data["Date_Time"] = pd.to_datetime(data["Date_Time"], format = "%Y-%m-%d %H:%M:%S")
data.index = data['Date_Time']
data.drop(["date"], axis = 1, inplace = True)
data["Next Date_Time"] = data["Date_Time"].shift(-1)
data

,open,high,low,close,volume,Time,Date_Time,Next Date_Time
Date_Time,,,,,,,,
2017-07-17 09:14:00,24011.40,24011.40,24011.40,24011.40,0,09:14:00,2017-07-17 09:14:00,2017-07-17 09:15:00
2017-07-17 09:15:00,24011.40,24014.40,23974.95,23974.95,0,09:15:00,2017-07-17 09:15:00,2017-07-17 09:16:00
2017-07-17 09:16:00,23975.45,23981.55,23963.15,23978.95,0,09:16:00,2017-07-17 09:16:00,2017-07-17 09:17:00
2017-07-17 09:17:00,23977.25,23995.30,23976.00,23984.60,0,09:17:00,2017-07-17 09:17:00,2017-07-17 09:18:00
2017-07-17 09:18:00,23983.25,23989.95,23983.25,23987.10,0,09:18:00,2017-07-17 09:18:00,2017-07-17 09:19:00
...,...,...,...,...,...,...,...,...
2024-03-07 15:25:00,47815.65,47815.65,47788.80,47802.45,0,15:25:00,2024-03-07 15:25:00,2024-03-07 15:26:00
2024-03-07 15:26:00,47805.65,47805.65,47790.45,47795.80,0,15:26:00,2024-03-07 15:26:00,2024-03-07 15:27:00
2024-03-07 15:27:00,47797.10,47802.95,47782.00,47800.35,0,15:27:00,2024-03-07 15:27:00,2024-03-07 15:28:00


In [5]:
def convert_timeframe(data, tf):
    data = data.copy()  # To avoid modifying the original DataFrame
    
    new_data = {
        "Date_Time": [],
        "Open": [],
        "High": [],
        "Low": [],
        "Close": []
    }
    
    step = 0
    for i in data.index:
        step += 1
        if step == 1:
            o = data.loc[i, "open"]
            t = i
            h = data.loc[i, "high"]
            l = data.loc[i, "low"]
        else:
            if data.loc[i, "high"] > h:
                h = data.loc[i, "high"]
            if data.loc[i, "low"] < l:
                l = data.loc[i, "low"]
            if step == tf or i.day != data.loc[i, "Next_Date_Time"].day:
                step = 0
                c = data.loc[i, "close"]
                new_data["Date_Time"].append(t)
                new_data["Open"].append(o)
                new_data["High"].append(h)
                new_data["Low"].append(l)
                new_data["Close"].append(c)
    
    return pd.DataFrame(new_data)


In [6]:
import pandas as pd

def convert_timeframe(data, tf):
    data = data.copy()  # To avoid modifying the original DataFrame
    
    new_data = {
        "Date_Time": [],
        "Open": [],
        "High": [],
        "Low": [],
        "Close": []
    }
    
    step = 0
    for i in range(len(data)):
        step += 1
        if step == 1:
            o = data.iloc[i]["open"]
            t = data.index[i]
            h = data.iloc[i]["high"]
            l = data.iloc[i]["low"]
        else:
            if data.iloc[i]["high"] > h:
                h = data.iloc[i]["high"]
            if data.iloc[i]["low"] < l:
                l = data.iloc[i]["low"]
            if step == tf or (i+1) == len(data) or data.index[i+1].day != data.index[i].day:
                step = 0
                c = data.iloc[i]["close"]
                new_data["Date_Time"].append(t)
                new_data["Open"].append(o)
                new_data["High"].append(h)
                new_data["Low"].append(l)
                new_data["Close"].append(c)
                if (i+1) < len(data):
                    t = data.index[i+1]
                    o = data.iloc[i+1]["open"]
                    h = data.iloc[i+1]["high"]
                    l = data.iloc[i+1]["low"]
    
    return pd.DataFrame(new_data)


In [7]:
data45 = convert_timeframe(data, 5)
data45_original = data45.copy()

In [8]:
data45.head()

,Date_Time,Open,High,Low,Close
0,2017-07-17 09:14:00,24011.40,24014.40,23963.15,23987.1
1,2017-07-17 09:19:00,23986.60,23998.25,23954.05,23982.9
2,2017-07-17 09:24:00,23983.55,23995.20,23967.95,23984.7
3,2017-07-17 09:29:00,23983.00,23984.85,23952.20,23954.9
4,2017-07-17 09:34:00,23954.15,23963.40,23921.20,23940.7


In [9]:
#data45.to_excel("5EMA_BUY5_BN.xlsx")

In [10]:
data45.to_csv("5EMA_5BN.csv")